In [1]:
import pandas as pd
import re

# Definovanie stĺpcov, pretože pôvodné dáta ich neobsahujú
columns = ['ID', 'Názov', 'Popis', 'Kategória', 'Cena', 'Recenzie']

# Načítanie dát z csv súboru
df = pd.read_csv('data.csv', encoding='utf-8', names=columns)

# Zobrazenie dát
print("Pôvodné Dáta:")
print(df)

# Funkcia na čistenie textu
def clean_text(text):
    text = text.lower()  # Zmena na malé písmená
    text = re.sub(r'[^\w\sá-žÁ-Ž]', '', text)  # Povolené znaky s diakritikou
    return text

# Aplikácia čistenia na stĺpce 'Názov' a 'Popis'
df['Názov'] = df['Názov'].apply(clean_text)
df['Popis'] = df['Popis'].apply(clean_text)

# Odstránenie duplicitných záznamov v prípade že existujú
df = df.drop_duplicates()

# Zobrazenie čistých dát
print("\nČisté Dáta:")
print(df)

Pôvodné Dáta:
   ID                              Názov  \
0   1   Televízia Samsung LED 155 palcov   
1   2       Televízia Sony LED 55 palcov   
2   3           Notebook Lenovo ThinkPad   
3   4                    Notebook MacAir   
4   5                    Notebook MacPro   
5   6  Chytrý telefón Samsung Galaxy S20   
6   7           Chytrý telefón iPhone 15   
7   8          Bezdrôtové slúchadlá Sony   
8   9       Bezdrôtové slúchadlá Pioneer   
9  10                  Slúchadlá Pioneer   

                                               Popis       Kategória     Cena  \
0  155-palcová televízia s LED technológiou a roz...       Televízia  1899.59   
1  55-palcová televízia s LED technológiou a rozl...       Televízia   799.99   
2  Tenko vrstvený notebook s výkonným procesorom ...        Notebook  1299.99   
3  Tenko vrstvený notebook s výkonným procesorom ...        Notebook  2277.88   
4  Notebook s výkonným procesorom Intel M2 a oper...        Notebook  4277.88   
5  Chytrý telef

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Spojenie 'Názov' a 'Popis' do jedného textového poľa
df['Text'] = df['Názov'] + " " + df['Popis']

# Inicializácia TF-IDF Vectorizéra
vectorizer = TfidfVectorizer()

# Vytvorenie TF-IDF matice
tfidf_matrix = vectorizer.fit_transform(df['Text'])

# Zobrazenie tvaru matice (počet dokumentov, počet unikátnych slov)
print(f"\nTF-IDF Matica má tvar: {tfidf_matrix.shape}")

# Príklad: Zobrazenie TF-IDF hodnôt pre prvý dokument
print("\nTF-IDF Hodnoty pre Prvý Dokument:")
print(tfidf_matrix[0].todense())


TF-IDF Matica má tvar: (10, 72)

TF-IDF Hodnoty pre Prvý Dokument:
[[0.         0.2501958  0.2501958  0.         0.         0.
  0.         0.         0.         0.2501958  0.         0.
  0.         0.         0.         0.         0.18607799 0.
  0.         0.         0.         0.21268929 0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.42537858 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.21268929
  0.         0.         0.         0.         0.09250244 0.
  0.         0.         0.21268929 0.         0.21268929 0.
  0.21268929 0.         0.         0.21268929 0.         0.42537858
  0.         0.         0.21268929 0.21268929 0.         0.
  0.         0.         0.         0.         0.21268929 0.        ]]


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Definovanie vstupov (TF-IDF matica) a cieľa (Kategória)
X = tfidf_matrix
y = df['Kategória']

# Rozdelenie dát na trénovaciu a testovaciu množinu (80% - 20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Inicializácia a trénovanie modelu
classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)

# Predikcia na testovacej množine
y_pred = classifier.predict(X_test)

# Výpočet presnosti
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPresnosť klasifikácie: {accuracy:.2f}")

# Podrobný report
print("\nReport klasifikácie:")
print(classification_report(y_test, y_pred, zero_division=0))


Presnosť klasifikácie: 0.50

Report klasifikácie:
              precision    recall  f1-score   support

    Notebook       0.00      0.00      0.00         0
   Slúchadlá       1.00      1.00      1.00         1
   Televízia       0.00      0.00      0.00         1

    accuracy                           0.50         2
   macro avg       0.33      0.33      0.33         2
weighted avg       0.50      0.50      0.50         2



In [13]:
import json

# Definovanie slovníka synoným
synonyms_dict = {
    "televízia": ["tv", "televízor", "telka"],
    "notebook": ["laptop", "počítač", "pc"],
    "chytrý telefón": ["smartphone", "mobil"],
    "slúchadlá": ["headphones", "earphones"]
}

# Funkcia na rozšírenie dotazu o synonymá
def expand_query(query):
    words = query.lower().split()
    expanded_words = []
    for word in words:
        if word in synonyms_dict:
            expanded_words.extend(synonyms_dict[word])
        expanded_words.append(word)
    return ' '.join(expanded_words)

# Test rozšírenia dotazu
query = "veľký notebook"
expanded_query = expand_query(query)
print(f"\nPôvodný dotaz: {query}")
print(f"Rozšírený dotaz: {expanded_query}")

# Uloženie slovníka do JSON súboru
with open('synonyms_dict.json', 'w', encoding='utf-8') as f:
    json.dump(synonyms_dict, f, ensure_ascii=False, indent=4)


Pôvodný dotaz: veľký notebook
Rozšírený dotaz: veľký laptop počítač pc notebook


In [14]:
# Generovanie fiktívnych užívateľských dotazov
user_queries = [
    "kúpiť televíziu Samsung",
    "najlepší notebook pre prácu",
    "lacné slúchadlá Sony",
    "chytrý telefón s podporou 5G",
    "televízia s rozlíšením 4K",
    "laptop s výkonným procesorom",
    "bezdrôtové slúchadlá s dlhou výdržou",
    "smartphone pre multitasking",
    "headphones s výborným zvukom",
    "počítač pre profesionálne využitie"
]

# Rozšírenie dotazov o synonymá
expanded_user_queries = [expand_query(q) for q in user_queries]

# Zobrazenie generovaných dotazov
print("\nGenerované Užívateľské Dotazy:")
for original, expanded in zip(user_queries, expanded_user_queries):
    print(f"Pôvodný: {original} --> Rozšírený: {expanded}")


Generované Užívateľské Dotazy:
Pôvodný: kúpiť televíziu Samsung --> Rozšírený: kúpiť televíziu samsung
Pôvodný: najlepší notebook pre prácu --> Rozšírený: najlepší laptop počítač pc notebook pre prácu
Pôvodný: lacné slúchadlá Sony --> Rozšírený: lacné headphones earphones slúchadlá sony
Pôvodný: chytrý telefón s podporou 5G --> Rozšírený: chytrý telefón s podporou 5g
Pôvodný: televízia s rozlíšením 4K --> Rozšírený: tv televízor telka televízia s rozlíšením 4k
Pôvodný: laptop s výkonným procesorom --> Rozšírený: laptop s výkonným procesorom
Pôvodný: bezdrôtové slúchadlá s dlhou výdržou --> Rozšírený: bezdrôtové headphones earphones slúchadlá s dlhou výdržou
Pôvodný: smartphone pre multitasking --> Rozšírený: smartphone pre multitasking
Pôvodný: headphones s výborným zvukom --> Rozšírený: headphones s výborným zvukom
Pôvodný: počítač pre profesionálne využitie --> Rozšírený: počítač pre profesionálne využitie


In [16]:
import scipy.sparse

# 1. Export klasifikovaných dát do CSV
df['Predikovaná Kategória'] = classifier.predict(X)
df.to_csv('classified_products.csv', index=False, encoding='utf-8')
print("\nKlasifikované dáta boli exportované do 'classified_products.csv'.")

# 2. Export TF-IDF matice do súboru
scipy.sparse.save_npz('tfidf_matrix.npz', tfidf_matrix)
print("TF-IDF matica bola uložená do 'tfidf_matrix.npz'.")

# 3. Export slovníka synonym do JSON
with open('synonyms_dict.json', 'w', encoding='utf-8') as f:
    json.dump(synonyms_dict, f, ensure_ascii=False, indent=4)
print("Slovník synonym bol uložený do 'synonyms_dict.json'.")

# 4. Export generovaných dotazov do textového súboru
with open('generated_user_queries.txt', 'w', encoding='utf-8') as f:
    for query in expanded_user_queries:
        f.write(query + '\n')
print("Generované užívateľské dotazy boli uložené do 'generated_user_queries.txt'.")


Klasifikované dáta boli exportované do 'classified_products.csv'.
TF-IDF matica bola uložená do 'tfidf_matrix.npz'.
Slovník synonym bol uložený do 'synonyms_dict.json'.
Generované užívateľské dotazy boli uložené do 'generated_user_queries.txt'.
